In [83]:
import folium
import pandas as pd
import geopandas as gpd
from folium import plugins

In [84]:
# Specify the path to the CSV file you want to load
csv_file_path = './deploy/MiniProject5_KMeans_newtable_v4.csv'

# Load the DataFrame from the CSV file
newtable = pd.read_csv(csv_file_path)

In [85]:
newtable.head()

,country,1972,1973,1974,1975,1976,1977,1978,1979,1980,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,0
0,Afghanistan,4.5,6.5,6.5,6.5,6.5,6.0,7.0,7.0,7.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,5.5,5.5,1
1,Albania,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3
2,Algeria,6.0,6.0,6.0,6.5,6.0,6.0,6.0,6.0,6.0,...,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,1
3,Andorra,3.5,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0
4,Angola,NaN,NaN,NaN,6.0,6.0,7.0,7.0,7.0,7.0,...,5.5,5.5,5.5,5.5,5.5,6.0,6.0,6.0,5.5,1


In [86]:
newtable.isnull().sum().to_frame()

,0
country,0
1972,51
1973,50
1974,48
1975,42
1976,41
1977,45
1978,42
1979,39
1980,38


In [87]:
# Load or create a dataset with country names and their corresponding latitude and longitude coordinates
geographic_data = pd.read_csv('./data/geographic_data.csv')

In [88]:
geographic_data[['latitude', 'longitude', 'country']] = geographic_data['latitude;longitude;country'].str.split(';', expand=True)

In [89]:
merged_data = pd.merge(newtable, geographic_data[['country', 'latitude', 'longitude']], on='country', how='left')

In [113]:
# Mark to companies'location
mymap = folium.Map(location=[100, 100], zoom_start=2)  # Specify a starting location and zoom level

In [114]:
# Merge the two datasets based on the 'country' column
merged_data = pd.merge(newtable, geographic_data, on='country', how='left')

In [115]:
print(newtable.columns)
print(geographic_data.columns)

Index(['country', '1972', '1973', '1974', '1975', '1976', '1977', '1978',
       '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987',
       '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '0'],
      dtype='object')
Index(['latitude;longitude;country', 'latitude', 'longitude', 'country'], dtype='object')


In [116]:
print(merged_data.head())

       country  1972  1973  1974  1975  1976  1977  1978  1979  1980  ...  \
0  Afghanistan   4.5   6.5   6.5   6.5   6.5   6.0   7.0   7.0   7.0  ...   
1      Albania   7.0   7.0   7.0   7.0   7.0   7.0   7.0   7.0   7.0  ...   
2      Algeria   6.0   6.0   6.0   6.5   6.0   6.0   6.0   6.0   6.0  ...   
3      Andorra   3.5   4.0   4.0   4.0   4.0   NaN   NaN   NaN   NaN  ...   
4       Angola   NaN   NaN   NaN   6.0   6.0   7.0   7.0   7.0   7.0  ...   

   2013  2014  2015  2016  2017  2018  0        latitude;longitude;country  \
0   6.0   6.0   6.0   6.0   5.5   5.5  1  3.393.911;67.709.953;Afghanistan   
1   3.0   3.0   3.0   3.0   3.0   3.0  3     41.153.332;20.168.331;Albania   
2   5.5   5.5   5.5   5.5   5.5   5.5  1      28.033.886;1.659.626;Algeria   
3   1.0   1.0   1.0   1.0   1.0   1.0  0      42.546.245;1.601.554;Andorra   
4   5.5   5.5   6.0   6.0   6.0   5.5  1     -11.202.692;17.873.887;Angola   

      latitude   longitude  
0    3.393.911  67.709.953  
1   41.153

In [117]:
# Convert the DataFrame to GeoJSON format
# You need to define the latitude and longitude columns in your DataFrame.
# For example, let's assume your DataFrame has 'latitude' and 'longitude' columns.
geojson_data = merged_data.to_dict(orient='records')

In [118]:
def colorPicker(population):
    if population < 1:
        return 'green'
    elif population >= 1 and population < 3:
        return 'orange'
    else:
        return 'red'

In [119]:
# Iterate through the rows of the DataFrame and add custom popups
for index, column in newtable.iterrows():
    country = column['country']
    population = column['0']  # Get the value from the last column

    folium.Marker(
        location=[0, 0],  # Fake coordinates (0, 0)
        icon=None,
        popup=f"{country}: {population}",
    ).add_to(mymap)


In [120]:
mymap